## Proyecto Final Visión por ordenador

#### Integrantes: María González Gómez y Jorge Vanco Sampedro

## 1: Cargamos las librerias necesarias

In [2]:
import os
import cv2
import imageio
import numpy as np
import pandas as pd
from typing import List
import matplotlib.pyplot as plt
from utils import *
from bow import BoW
from dataset import Dataset
from image_classifier import ImageClassifier
import time
from tqdm import tqdm
import sys
import pickle

## 2: Carga del dataset

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("ahemateja19bec1025/traffic-sign-dataset-classification")

print("Path to dataset files:", path)


Path to dataset files: C:\Users\meryg\.cache\kagglehub\datasets\ahemateja19bec1025\traffic-sign-dataset-classification\versions\2


In [ ]:
## código usado por si fuera necesario renombrar carpetas 

# import shutil
# num_to_label = pd.read_csv("../data/labels.csv", index_col="ClassId").to_dict()["Name"]
# num_to_label = {str(k): v for k,v in num_to_label.items()}
# num_to_label = {str(k): v.replace('/', '') for k, v in num_to_label.items()}
# print(num_to_label)

# train_data_dir = "../data/traffic_Data/DATA/"
# for dir in os.listdir(train_data_dir):
#     if dir in num_to_label.keys():
#         dir_path = os.path.join(train_data_dir, dir)
#         new_dir_path = os.path.join(train_data_dir, num_to_label[dir])
#         print(f"Renaming {dir_path} to {new_dir_path}")
#         if os.path.exists(dir_path):
#             if os.path.exists(new_dir_path):
#                 shutil.move(dir_path, new_dir_path)
#             else:
#                 os.renames(dir_path, new_dir_path)
#         else:
#             print(f"Path {dir_path} does not exist")

# test_data_dir = "../data/traffic_Data/TEST/"

# for image in os.listdir(test_data_dir):
#     num = image[:3]
#     try:
#         num = str(int(num))
#         label = num_to_label[num]
#         dir_path = os.path.join(test_data_dir, label)
#         if not os.path.exists(dir_path):
#             os.makedirs(dir_path)

#         image_path = os.path.join(test_data_dir, image)
#         shutil.move(image_path, dir_path)
#     except:
#         pass

## 3: Train Test Split

In [4]:
# Cargar conjuntos de datos
training_set = Dataset.load("../data/traffic_Data/DATA", "*png")
validation_set = Dataset.load("../data/traffic_Data/TEST", "*png")

print(training_set[0])
print(validation_set[0])

../data/traffic_Data/DATA\Bicycles crossing\030_0001.png
../data/traffic_Data/TEST\Bicycles crossing\030_0001_j.png


## 4: Creación del extractor de carácterísticas SIFT

In [ ]:
# Crear el extractor de características SIFT
feature_extractor = cv2.SIFT_create()

# Extraer descriptores
print("\nComputing SIFT descriptors...")
time.sleep(0.1)  # Previene problemas de concurrencia entre tqdm y print

descriptors = []  # Lista para almacenar descriptores
for path in tqdm(training_set, unit="image", file=sys.stdout):
    # Cargar la imagen en escala de grises
    image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    try:
        # Detectar y describir características SIFT
        keypoints, descriptor = feature_extractor.detectAndCompute(image, None)
        if descriptor is not None:
            descriptors.append(descriptor)
        else:
            print(f"Advertencia: No se encontraron descriptores en {path}")
    except Exception as e:
        print(f"Error procesando {path}: {e}")
print(f"Número de descriptores extraídos: {len(descriptors)}")


## 5: Creación del vocabulario con K-Means

In [ ]:
# Define constants
vocabulary_size = 200
iterations = 20
termination_criteria = (cv2.TERM_CRITERIA_MAX_ITER | cv2.TERM_CRITERIA_EPS, iterations, 1e-6)

# Initialize BOWKMeansTrainer
words = cv2.BOWKMeansTrainer(vocabulary_size, termination_criteria)

# Add descriptors to the trainer
for desc in tqdm(descriptors, desc="Adding descriptors"):
    words.add(desc)

time.sleep(0.1)  # Prevent tqdm printing issues
print("\nClustering descriptors into", vocabulary_size, "words using K-means...")

# Perform k-means clustering to build the vocabulary
vocabulary = words.cluster()
print(f"Vocab: {vocabulary}")
filename=  "vocabulary.pickle"
# TODO: Open the file from above in the write and binay mode
with open(filename,"wb") as f:
    pickle.dump(["SIFT", vocabulary], f, pickle.HIGHEST_PROTOCOL)
    
try:
    with open("vocabulary.pickle", "rb") as f:
        data = pickle.load(f)
        print(f"Data from vocab.pickle: {data}")
except Exception as e:
    print(f"Error al cargar el archivo: {e}")

## 6: Inferencia en Train

In [ ]:
# Load vocabulary into BoW
bow = BoW()
bow.load_vocabulary(filename.strip(".pickle"))

# Train the image classifier
image_classifier = ImageClassifier(bow)
print(f"Image classifier: {image_classifier}")
image_classifier.train(training_set, iterations)

# Save the trained classifier
classifier = "classifier"
image_classifier.save(classifier)

print("Vocabulary and classifier successfully built and saved.")


print("Empezando el train: ")

bow = BoW()
# TODO: Especify the args for the loading method
bow.load_vocabulary(filename.strip(".pickle"))

image_classifier = ImageClassifier(bow)
# TODO: Especify the args for the loading method
image_classifier.load(classifier)
# TODO: Especify the args for the loading method
image_classifier.predict(training_set)


## 7: Inferencia en Test

In [ ]:
print("Empezando el test: ")
bow = BoW()
# TODO: Especify the args for the loading method
bow.load_vocabulary(filename.strip(".pickle"))

image_classifier = ImageClassifier(bow)
# TODO: Especify the args for the loading method
image_classifier.load(classifier)
# TODO: Especify the args for the loading method
image_classifier.predict(validation_set)

## 8: Tracking de señales

## 9: Clasificación de señales

In [ ]:
# Carga tu modelo SVM entrenado
import joblib
svm_model = joblib.load("classifier_model.xml.pkl")

# Preprocesa el ROI y extrae características (ejemplo con HOG)
hog = cv2.HOGDescriptor()
features = hog.compute(cv2.resize(roi, (64, 64)))

# Clasifica la señal
prediction = svm_model.predict(features.reshape(1, -1))
print(f"Señal detectada: {prediction[0]}")
